## Import modules and define constants

In [1]:
# if this is a py file, it's generated from notebook (ipynb file)

import shutil
from pathlib import Path
from datetime import date, timedelta

digest_start = date(2020, 3, 1)
start_year = 2020
cur_year = 2021
one_day = timedelta(days=1)

 # not using strftime("%B") for months since it depends on the current locale
month_en = ('January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December')
weekday_zh_hant = ('週一', '週二', '週三', '週四', '週五', '週六', '週日')

## Get manual md path and content

In [2]:
def path_manual(y, m, d):
    return Path('manual', str(y), str(m), f'{d}.md')

def manual_content(y, m, d):
    md_file = path_manual(y, m, d)
    if Path(md_file).is_file():
        with open(md_file, 'r') as f:
            content = f.read()
        return content
    else:
        return ""
# manual_content(2021,3,1)

## Generate the starting string (for mdx that will be generated)

In [3]:
def start_str(y, m, d):
    '''Return the starting string for the page of y/m/d'''
    dt = date(y, m, d)

    front_matter = f'''---
title: {m} 月 {d} 號（{weekday_zh_hant[dt.weekday()]}）
description: {y} 年 {m} 月 {d} 日法國新冠肺炎疫情匯報。法國 COVID-19 日誌第 {(dt - digest_start).days + 1} 篇。
---
'''
    export_import = f'''
export const date = "{dt.strftime('%Y-%m-%d')}";
import {{ Subtitle, Grace, Fish, OfficialData, SourceFb, Figure, ChartCases }} from "@site/src/scripts/components";
'''
    subtitle = '''
<Subtitle date={date} />
'''
    sourceOfData = '''
<div className="comment--translc_gray">📈 數據來源：請參考附錄<a href="../../sources">資料來源</a>。</div>
'''
    chartCases = '''
<ChartCases date={date} />
'''
    official_data='''
## 法國官方數據 {#official-data}

<OfficialData date={date} />
'''
    main_img = '''
<Figure date={date}></Figure>
'''
    news_header = '''
<SourceFb date={date} />

## 本日新聞重點 {#new}
'''
    news_header = news_header + "\n" if path_manual(y, m, d).is_file() else ""

    if dt == date(2020,3,1):
        return  front_matter + export_import + subtitle + main_img
    else:
        return front_matter + export_import + subtitle + main_img + sourceOfData + chartCases + official_data + news_header
# print(start_str(2020,3,2))

## Copy `manual/{cur_year}/intro.md` to `docs/{cur_year}/intro.md`

In [4]:
def create_intro_md():
    '''Copy `manual/{cur_year}/intro.md` to `docs/{cur_year}/intro.md`. '''
    manual_intro_md = Path('manual', str(cur_year), 'intro.md')
    docs_intro_dir = Path('..', '..', 'docs', str(cur_year))
    docs_intro_md = Path(docs_intro_dir, 'intro.md')

    if not manual_intro_md.is_file():
        raise Exception(f"File `manual/{cur_year}/intro.md` doesn't exist, can't copy it to `docs/{cur_year}/intro.md`.")
    else:
        docs_intro_dir.mkdir(parents=True, exist_ok=True)
        shutil.copy(manual_intro_md, docs_intro_md)
        print(f'Copied `manual/{cur_year}/intro.md` to `docs/{cur_year}/intro.md`.')
# create_intro_md()

## Generate digest pages

From `manual/{year}/{month}/{day}.md`, generate `docs/{year}/{month_en_lower}/{day}.mdx`

Note: Should generate pages ONLY for dates after 2020/3/1 (included)

In [5]:
def generate_a_page(y, m, d):
    folder = Path('..', '..', 'docs', str(y), month_en[m-1].lower())
    file_name = f'{d}.mdx'
    mdx = Path(folder, file_name)

    man = manual_content(y, m, d)
    mdx_content = start_str(y, m, d) + man # start_str will make sure the input `y, m, d` is of right format

    prompt_note = "" if man else " (without manual content)"
    prompt = f'> created mdx for {y}/{m}/{d} in `docs`{prompt_note}'
    if Path(mdx).is_file():
        prompt = f'> overwrote mdx of {y}/{m}/{d} in `docs`{prompt_note}'

    # write file
    folder.mkdir(parents=True, exist_ok=True)
    with open(mdx, 'w') as f:
        f.write(mdx_content)

    # print prompt
    print(prompt)

def generate_pages(s_y, s_m, s_d, e_y, e_m, e_d):
    s_dt = date(s_y, s_m, s_d)
    e_dt = date(e_y, e_m, e_d)
    n_pages = (e_dt - s_dt).days + 1
    if n_pages < 2:
        raise Exception("The ending date should be at least one day after the starting day.")

    prompt = f'writing {n_pages} pages: from {s_y}/{s_m}/{s_d} to {e_y}/{e_m}/{e_d}'
    print(f'Start {prompt}...')

    dt = s_dt
    while (dt != e_dt + one_day):
        generate_a_page(dt.year, dt.month, dt.day)
        dt += one_day

    print(f'Finish {prompt}.')

## Clear generated pages

Delete folders `docs/{year}/` and all their contents

In [6]:
def clear_generated_pages():
    print('Start clearing generated files...')
    for y in range(start_year, cur_year + 2):
        dir_to_delete = Path('..', '..', 'docs', str(y))
        if dir_to_delete.is_dir():
            shutil.rmtree(dir_to_delete)
            print(f"> deleted folder `{y}` in `docs` (as well as all its contents)")
    print('Finish clearing generated files.')
# clear_generated_pages()

## Execute the `main` function

In [7]:
def main():
    clear_generated_pages()
    generate_a_page(2020,12,31)
    generate_a_page(2020,3,1)
    generate_a_page(2020,4,1)
    generate_a_page(2020,5,1)
    generate_a_page(2020,6,1)
    generate_a_page(2020,7,1)
    generate_a_page(2020,8,1)
    generate_a_page(2020,9,1)
    generate_a_page(2020,10,1)
    generate_a_page(2020,11,1)
    generate_a_page(2020,12,1)
    generate_a_page(2021,1,1)
    generate_a_page(2021,2,1)
    generate_a_page(2021,3,1)
    generate_a_page(2021,4,1)
    generate_pages(2021,3,2,2021,3,5)
    # generate_pages(2020,3,1,2021,3,29)
    create_intro_md()
    
main()

Start clearing generated files...
> deleted folder `2020` in `docs` (as well as all its contents)
> deleted folder `2021` in `docs` (as well as all its contents)
Finish clearing generated files.
> created mdx for 2020/12/31 in `docs`
> created mdx for 2020/3/1 in `docs` (without manual content)
> created mdx for 2020/4/1 in `docs` (without manual content)
> created mdx for 2020/5/1 in `docs` (without manual content)
> created mdx for 2020/6/1 in `docs` (without manual content)
> created mdx for 2020/7/1 in `docs` (without manual content)
> created mdx for 2020/8/1 in `docs` (without manual content)
> created mdx for 2020/9/1 in `docs` (without manual content)
> created mdx for 2020/10/1 in `docs` (without manual content)
> created mdx for 2020/11/1 in `docs`
> created mdx for 2020/12/1 in `docs`
> created mdx for 2021/1/1 in `docs`
> created mdx for 2021/2/1 in `docs`
> created mdx for 2021/3/1 in `docs`
> created mdx for 2021/4/1 in `docs` (without manual content)
Start writing 4 page